# 資料正規化與標準化

### Why normalization and standardization?

為什麼要這樣做？真的有關係嗎？不標準化有什麼影響？
>在與我們的研發團隊成員交談並研究在線文章時，我很快意識到標準化多變量數據非常重要，特別是當變量尺度顯著不同時。當使用多尺度變量時，在多變量分析期間影響模型穩定性和參數估計精度。例如，在邊界檢測中，範圍在0到100之間的變量將超過範圍在0和1之間的變量。使用沒有標準化的變量可以在分析中為更大範圍的變量賦予更大的重要性。將數據轉換為可比較的比例可以防止這個問題。在神經網絡中標準化連續預測變量非常重要。在執行回歸分析時，標準化多尺度變量有助於減少包含交互項的模型的多重共線性問題。在集群分析之前標準化數據也非常關鍵。聚類是一種無監督學習技術，可將觀察分類為類似的組或聚類。常用的相似度量是歐幾里德距離。歐幾里德距離的計算方法是取觀察值之差的平方和的平方根。變量之間的比例差異會極大地影響該距離。通常，具有大差異的變量對該度量的影響大於具有小差異的變量。因此，在執行群集之前，建議標準化多比例變量。標準化或標準化數據在主成分分析（PCA）中也很重要，因為它將原始數據投影到正交方向上，從而最大化方差。但是，基於樹的分析對異常值不敏感，不需要變量轉換。因此，決策樹，隨機森林和/或梯度增強算法不需要標準化多尺度數據。<br><br>by Sas官方技術文章

資料正規化與標準化在機器學習流程當中相當重要，許多機器學習算法的特性需要透過資料的縮放方能表現得更好。有兩種主要的資料縮方法分別為:正規化、標準化。在本筆記本中，將探討的主題為:
<li>正規化資料
<li>標準化資料
<li>正規化與標準化的使用時機
<li>機器學習模型及適合的資料縮放方法

## 資料集
在此文章內，我使用Pima Indians Diabetes Dataset。該資料出自於Kaggle的公開資料科學比賽，比賽的內容為預測5年內發生糖尿病的機率。
檔案的目錄在此資料夾中的data資料夾內，檔案名稱為pima-indians-diabetes.csv。

### 1 正規化
<p>
正規化有數種不同的方式，此案例中的正規化是將計量資料的數值轉換為0-1之間的數字。用途是在訓練機器學習模型時為了避免因為資料尺度不同，某些資料的數值表現較其他為大，而左右了某些對資料計量尺度較為敏感的機器學習模型；使其學習時偏頗於數值較大的資料而忽略其他數值較小的。</p><p>
例子:SVM模型的特點是以最大化資料點之間的距離，假如特徵中的其中一個有著比其他特徵尺度都大的數值，則此特徵支配其他特徵對模型的影響。將所有特徵都縮放到[0, 1]的尺度，則所有特徵的影響相等。</p>

透過正規化以後不同尺度的資料將被統一。例如:身高、體重皆為計量單位，但因為尺度不同，計量大小的標準也不一樣。透過正規化，尺度統一為0-1之間的數字，因此無論是身高或體重，都可以用同樣的尺度來表示計量的多寡。在做正規化之前先決條件是我們已經知道資料的最大值、最小值，有時候需要透過觀察資料得知、有時候則需要一些特殊領域的知識，如:用來表現一張8bit圖片資料的多維陣列，最小值是0、最大值是255，因為8bit圖片顧名思義是由0-255的非負整數的像素組成，因此我們能夠使用此8bit照片的特性來做正規化。正規化的公式為:

\begin{align}
scaledvalue = \frac{value - min}{max - min}
\end{align}


### 1-1 Python implementation

In [71]:
from csv import reader

def load_csv(path):
    
    dataset = []
    
    with open(path, 'r') as f:
        csv_reader = reader(f)
        
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
            
    return dataset


def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
        

def min_max(dataset):
    min_max_output = []
    
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        _min = min(col_values)
        _max = max(col_values)
        min_max_output.append((_min, _max))
        
    return min_max_output


def normalize_dataset(dataset, dataset_min_max):
    for column in range(len(dataset[0])):
        _min, _max = dataset_min_max[column]
        for row in dataset:
            row[column] = (row[column] - _min) / (_max - _min)

In [2]:
dataset = load_csv('data/pima-indians-diabetes.data.csv')
print(dataset[0])

for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)   
print(dataset[0])

dataset_min_max = min_max(dataset)
print(dataset_min_max)

normalize_dataset(dataset, dataset_min_max)
print(dataset[0])

['6', '148', '72', '35', '0', '33.6', '0.627', '50', '1']
[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0, 1.0]
[(0.0, 17.0), (0.0, 199.0), (0.0, 122.0), (0.0, 99.0), (0.0, 846.0), (0.0, 67.1), (0.078, 2.42), (21.0, 81.0), (0.0, 1.0)]
[0.35294117647058826, 0.7437185929648241, 0.5901639344262295, 0.35353535353535354, 0.0, 0.5007451564828614, 0.23441502988898377, 0.48333333333333334, 1.0]


### 1-2 Pandas implementation

In [4]:
import pandas as pd

def normalized_df(df, cols):
    """
    Normalize dataframe by given columns.
    
    parameters:
    df(pandas.DataFrame):The input dataframe. 
    cols(list):A list of column names that need to be normalized.
    
    return:
    df(pandas.DataFrame):The normalized dataframe.
    """
    df = df.copy()
    
    for col in cols:
        df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
        
    return df


#load Pima Indians Diabetes Dataset
data = pd.read_csv('data/pima-indians-diabetes.data.csv')
#normalize all columns
normalized_data = normalized_df(data, data.columns)

normalized_data.describe()

,6,148,72,35,0,33.6,0.627,50,1
count,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000
mean,0.226014,0.607333,0.566407,0.207248,0.094449,0.476758,0.168093,0.203651,0.348110
std,0.198287,0.160696,0.158755,0.161152,0.136268,0.117572,0.141545,0.195872,0.476682
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.058824,0.497487,0.508197,0.000000,0.000000,0.406855,0.070666,0.050000,0.000000
50%,0.176471,0.587940,0.590164,0.232323,0.037825,0.476900,0.125107,0.133333,0.000000
75%,0.352941,0.703518,0.655738,0.323232,0.150709,0.545455,0.233561,0.333333,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### 2 標準化
標準化將資料後，資料的算術平均數為0，標準差為1。作法是以資料的算術平均數作為中心點對資料進行縮放，因此透過標準化後資料呈現了高斯分布。進行資料標準化的先決條件是每個欄位的算術平均數以及標準差。

### 2-1 Python implementation

##### 2-1-1 算數平均數
算術平均數是加總所有數字後，除以資料數量所得到數字。
\begin{align}
mean = \frac{\sum_{i=1}^nvalues_i}{n}
\end{align}

In [11]:
def column_means(dataset):
    means = []
    
    for column in range(len(dataset[0])):
        column_values = [row[column] for row in dataset]
        column_mean = sum(column_values) / float(len(column_values))
        means.append(column_mean)
        
    return means

#### 2-1-2 標準差
標準差是敘述資料對其算術平均數的平均離散程度，也可理解為每個資料點對資料之算術平均數的平均距離。
\begin{align}
std = \sqrt\frac{\sum_{i=1}^n(values_i - mean)^2}{n - 1}
\end{align}
    對於標準差公式的分母為什麼是n-1而不是n，這個<a href='https://www.youtube.com/watch?v=9ONRMymR2Eg'>影片</a>很好的解釋了其背後的原理，對於數學證明的過程也很清楚，強烈建議觀賞<a href='https://www.youtube.com/watch?v=D1hgiAla3KI'>推導影片</a>。簡而言之，計算樣本方差的時候常會低估，因此n-1可以平衡這個現象。

In [22]:
from math import sqrt

def column_stds(dataset, column_means):
    stds = []
    
    for column in range(len(dataset[0])):
        varience = sum([pow((row[column] - column_means[column]), 2) for row in dataset])
        std = sqrt(varience / (len(dataset) - 1))
        stds.append(std)
        
    return stds

#### 2-1-3 標準化
標準化將數字以算術平均數為中心點，重新縮放資料以後的數字在一個標準差內。
\begin{align}
standardlization = \frac{values_i - mean}{std}
\end{align}

In [39]:
def column_standardlization(dataset, column_means, column_stds):
    standardlizations = []
    
    for column in range(len(dataset[0])):
        for row in dataset:
            row[column] = (row[column] - column_means[column]) / column_stds[column]

In [47]:
dataset = load_csv('data/pima-indians-diabetes.data.csv')
print(dataset[0])

for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)   
print(dataset[0])

dataset_means = column_means(dataset)
print(dataset_means)

dataset_stds = column_stds(dataset, dataset_means)
print(dataset_stds)

column_standardlization(dataset, dataset_means, dataset_stds)
print(dataset[0])

['6', '148', '72', '35', '0', '33.6', '0.627', '50', '1']
[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0, 1.0]
[3.8450520833333335, 120.89453125, 69.10546875, 20.536458333333332, 79.79947916666667, 31.992578124999977, 0.4718763020833327, 33.240885416666664, 0.3489583333333333]
[3.3695780626988623, 31.97261819513622, 19.355807170644777, 15.952217567727677, 115.24400235133837, 7.8841603203754405, 0.33132859501277484, 11.76023154067868, 0.4769513772427971]
[0.6395304921176576, 0.8477713205896718, 0.14954329852954296, 0.9066790623472505, -0.692439324724129, 0.2038799072674717, 0.468186870229798, 1.4250667195933604, 1.3650063669598067]


### 2-2 Pandas implementation

In [64]:
def standardlized_df(df, cols):
    """
    Standardlize dataframe by given columns.
    
    parameters:
    df(pandas.DataFrame):The input dataframe. 
    cols(list):A list of column names that need to be Standardlized.
    
    return:
    df(pandas.DataFrame):The standardlized dataframe.
    """
    df = df.copy()
    
    for col in cols:
        df[col] = (df[col] - df[col].mean()) / df[col].std()
        
    return df


#load Pima Indians Diabetes Dataset
data = pd.read_csv('data/pima-indians-diabetes.data.csv')
#standardlize all columns
standardlized_dataset = standardlized_df(data, data.columns)

standardlized_dataset.describe()

,6,148,72,35,0,33.6,0.627,50,1
count,7.670000e+02,7.670000e+02,7.670000e+02,7.670000e+02,7.670000e+02,7.670000e+02,7.670000e+02,7.670000e+02,7.670000e+02
mean,-2.779176e-17,6.716343e-17,-2.570738e-16,1.852784e-17,-2.315980e-17,5.604672e-16,-2.547578e-17,1.806465e-16,4.168764e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.139835e+00,-3.779393e+00,-3.567800e+00,-1.286043e+00,-6.931069e-01,-4.055028e+00,-1.187563e+00,-1.039715e+00,-7.302765e-01
25%,-8.431761e-01,-6.835597e-01,-3.666686e-01,-1.286043e+00,-6.931069e-01,-5.945530e-01,-6.883133e-01,-7.844455e-01,-7.302765e-01
50%,-2.498586e-01,-1.206809e-01,1.496428e-01,1.555967e-01,-4.155294e-01,1.206426e-03,-3.036949e-01,-3.589967e-01,-7.302765e-01
75%,6.401176e-01,5.985530e-01,5.626920e-01,7.197165e-01,4.128660e-01,5.842901e-01,4.625254e-01,6.620804e-01,1.367559e+00
max,3.903364e+00,2.443544e+00,2.731200e+00,4.919275e+00,6.645349e+00,4.450388e+00,5.877350e+00,4.065671e+00,1.367559e+00


### 更多的資料縮放方法
資料縮放的方法多種多樣，以下羅列出更多的資料縮放方法:
<li>資料正規化，但數值介於-1和1之間
<li>標準化資料，但是標準差在1以上
<li>指數變換，如對數，平方根和指數
<li>諸如Box-Cox之類的功率變換用於校正資料中的偏態分布。